In [1]:
!pip install flask
!pip install sqlalchemy
!pip install flask_cors
!pip install flask-restful
!pip install supabase_py

!pip install radvel
!pip install numpy
!pip install lightkurve
!pip install astropy
!pip install glob
# !pip install psycopg2-binary
# !pip install thirdweb-sdk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.1/133.1 KB 28.4 MB/s eta 0:00:00
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 2.0.0
    Not uninstalling markupsafe at /shared-libs/python3.7/py-core/lib/python3.7/site-packages, outside environment /root/venv
    Can't uninstall 'MarkupSafe'. No files were found to uninstall.
  Attempting uninstall: Jinja2
    Found existing installation: Jinja2 2.11.3
    Not uninstalling jinja2 at /shared-libs/python3.7/py-core/lib/python3.7/site-packages, outside environment /root/venv
    Can't uninstall 'Jinja2'. No files were found to uninstall.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy 3.4.2 requires typing-extensions<4.2.0,>=3.7.4; python_version < "3.8", but you have typing-extensions 4.4.0 which

In [2]:
import radvel
import numpy as np
import pandas as pd
import lightkurve as lk
import astropy.io.fits as pf
from glob import glob as glob
import matplotlib.pyplot as plt

/root/venv/lib/python3.7/site-packages/radvel/gp.py:33: ImportWarning: celerite not installed. GP kernals using celerite will not work. Try installing celerite using 'pip install celerite'
  Try installing celerite using 'pip install celerite'", ImportWarning)


# Radial Velocities

Radial velocities allow us to measure the mass of the planet by studying the light of the star. If we split the light of a star into all of its components (all of the colours of the rainbow) we see some of light missing as dark lines. As the planet orbits around the star, the exact location of these dark lines shifts (due to the star moving in a tiny little circle). Therefore, by measuring the exact location of these dark lines, we can determine the motion of the star caused by the orbitin planet. These measuremens are called radial velocity measurents. 


Notes:

- You need to run each cell. Do this by either pressing the 'run' button at the top of the page or pressing shift+enter (this is what Nora does in the videos). 

- When you runa cell, a star will appear in the brackets to the left of the cell while the cell is runnning. Once it has successfully completed running the code the star will turn into a number (often it runs so fast that you don't see the star).

- Text following a # is ignored by the code, so all comments are shown following # (except in this cell because I made this into a text cell)

by *Nora Eisner*

### first we need some data! 

A good place to look for radial velocity data of a given target is in the ESO exoplant archive: 
http://archive.eso.org/scienceportal/home
(note that only southern targets are liisted in the ESO archive and not all of the data is publicly available). 

NOTE: once you have downloaded the ZIP file, click on all of the files that end in '.tar' to 'untar' them. This should create folders called 'data', 'data 1' etc 

In [3]:
# first we'll look at the TESS data 

TIC =  'TIC 266980320'

# search what SPOC data is available for a given target - and chose a sector
sector_data = lk.search_lightcurve(TIC, author = 'SPOC', exptime = 120) # !! CHANEG THIS (see above)

lc = sector_data[0].download()

fig, ax = plt.subplots(figsize = (8,4))
%matplotlib inline
lc.plot(ax = ax, lw = 0, marker = '.', color = 'k')


<AxesSubplot:xlabel='Time - 2457000 [BTJD days]', ylabel='Flux [$\\mathrm{e^{-}\\,s^{-1}}$]'>

In [4]:
#look for the data on the ESO archive and download it as a ZIP file

In [5]:
# IMPORTANT! define the path to where you stored the data: e.g. /Users/Nora/Downloads/test_case

path = '/Users/Nora/Downloads/test_files/'

# ^ IMPORTANT TO CHANGE! ^

files = glob('{}*.fits'.format(path))


In [6]:
# let's open just one of the spectra and have a loook at what a spectra look like

# change this path to where you have the files saved!
# the zero just means that we're plotting the first file but you can plot any of the spectra
data = pf.open(files[0])

# plot
fig, ax = plt.subplots(figsize = (8,5))

plt.plot(data[1].data['WAVE'][0],data[1].data['FLUX'][0]/4000, color = 'k')

plt.xlim(5160,5190)
plt.ylim(0,2)
ax.tick_params(axis="y",direction="inout", labelsize = 16) #, pad= -20)
ax.tick_params(axis="x",direction="inout", labelsize = 16) #, pad= -17)   
ax.tick_params(axis='both', length = 7, left='on', top='on', right='on', bottom='on')

ax.set_xlabel(r'Wavelength ($\mathrm{\AA})$', fontsize = 16)
ax.set_ylabel('Intensity', fontsize = 16)

plt.tight_layout()

IndexError: list index out of range

The extracted radial velocities are stored in the data files. these files are known as 'fits' files and can be a little tricky to work with. Let's start by opening just one file and printing the radial velocity value!

In [0]:
files = glob('{}/data*/*/*/*ccf_G2_A.fits'.format(path))

# open file: 

data = pf.open(files[5])

# get the information stored in this file
header = data[0].header

header


You can see that that there is a lot of of information on this file, inlduding when this observation was taken, where it was taken, by whome it was taken, the location of the target and much more! For plotting the radial velocity curve, the most important thing that we need to know is the radial velocity value and the time the observation was taken!

In [0]:
time = header['HIERARCH ESO DRS BJD']
rv = header['HIERARCH ESO DRS CCF RVC']

Alone, these values don't tell us much, so instead let us extract the radial velocities and times from all of the available data files and plot them all together. 

In [0]:
# make some empty list tha we can fill with the information one file at a time

time_list = []
rv_list = []

for file in files:
    try:
        #open file
        data = pf.open(file)
        #get the information stored in this file
        header = data[0].header
        
        time = header['HIERARCH ESO DRS BJD']
        rv = header['HIERARCH ESO DRS CCF RVC'] * 1000 # convert to m/s
        
        # add the values to the lists
        time_list.append(time)
        rv_list.append(rv)
    except:
        continue

In [0]:
# let's plot the data
rv_list = rv_list - np.median(rv_list)

fig, ax = plt.subplots(figsize = (8,4))

plt.scatter(time_list, rv_list)

plt.xlabel("Time (BJD)")
plt.ylabel("RV (m/s)")

In [0]:
# phase fold the RVs at the period of the planet
# from this paper: https://www.aanda.org/articles/aa/pdf/2019/03/aa34853-18.pdf

period = 6.03607
T0 = 8329.1996 + 2450000 # add the offset from the TESS dates


phased_time = np.array([( ( t - T0*period) % period) / period for t in time_list])


# let's plot the data phase folded

fig, ax = plt.subplots(figsize = (8,4))
plt.scatter(phased_time, rv_list)
plt.xlabel("Phase")
plt.ylabel("RV (m/s)")

In [0]:
# we will now fit a simlpe radial velocity curve to this
# the values needed for this fit can be obtained from the published paper: https://www.aanda.org/articles/aa/pdf/2019/03/aa34853-18.pdf

# function 
def get_rv_curve_single(time_array, t0, P, ecc, omega,i,gamma, k):
    
    true_anom = radvel.orbit.true_anomaly(time_array, t0, P, ecc)
    
    rv = gamma + k *( ecc*np.cos(omega) + np.cos(true_anom+omega) )
    
    return rv



In [0]:
# parameters needed to fit the R model - all of these values can be found in the paper

ecc = 0               # eccentricity (how circular the orbit is (0 = circular))
omega = np.pi/2             # in radians
i = 86.38             # inclination of the orbit
gamma = 0             # offset
k = 6.17              # the amplitude of the RV signal (how much wobble)


In [0]:
# determine the time to make the model over: 

time_array = np.linspace(np.nanmin(time_list),np.nanmax(time_list),1000)

rv_curve = get_rv_curve_single(time_array, T0, period, ecc, omega,i,gamma, k)

In [0]:
# let's plot the data with the model

fig, ax = plt.subplots(figsize = (8,4))

plt.scatter(time_list, rv_list)
plt.plot(time_array, rv_curve, color = 'orange')
plt.xlabel("Time (BJD)")
plt.ylabel("RV (m/s)")

In [0]:
# phase fold the RVs at the period of the planet
# from this paper: https://www.aanda.org/articles/aa/pdf/2019/03/aa34853-18.pdf


phased_model= np.array([( ( t - T0*period) % period) / period for t in time_array])

# let's plot the data phase folded
fig, ax = plt.subplots(figsize = (8,4))

plt.scatter(phased_model, rv_curve, color = 'orange', s = 4, label = 'RV model')
plt.scatter(phased_time, rv_list)

plt.legend()
plt.xlabel("Phase")
plt.ylabel("RV (m/s)")

In [0]:
# given the amplitude of the the curve and the mass of the star, we can then work out the mass of the planet!

In [0]:
import astropy.units as u
import astropy.constants as c

mass_star = 0.92*u.Msun # solar masses

def det_mass(mass_star, k, period):
    
    period = period*u.day
    k = k*u.m/u.s
    return (((mass_star ** 2) * period) / (2 * np.pi * c.G)) ** (1./3.) * k 
    
    

In [0]:
planet_mass = det_mass(mass_star, k, period).to('Mearth')

print ("The mass of the planet is approximately {}".format(planet_mass))

In [0]:
Lightcurve Manipulation


[1]






















#%matplotlib notebook
# github.com/signal-k/polygon
!pip install lightkurve

import matplotlib.pyplot as plt
import lightkurve as lk






































This output has been hidden. Show it.







Star selection


[2]


















TIC = 'TIC 284475976' # TIC Star ID
sector_data = lk.search_lightcurve(TIC, author = 'SPOC', sector = 23) # can remove each arg if needed
sector_data
lc = sector_data.download()
lc.plot()














































 







[3]










lc.plot(linewidth = 0, marker = '.', color = 'lightcyan', alpha = 0.3)




































 









Plotting from multiple sectors


[4]














TIC_2 = 'TIC 55525572'
available_data_all = lk.search_lightcurve(TIC_2, author = 'SPOC')
available_data_all




































 







[5]












select_sector = available_data_all[0:4]
select_sector




































 







[6]












lc_collection = select_sector.download_all() # download all the sectors ([0:4])
lc_collection




































 







[7]










lc_collection.plot(linewidth = 0, marker = '.')




































 






Normalise graph points


[8]












lc_collection_stitched = lc_collection.stitch()
lc_collection_stitched.plot(linewidth = 0, marker = '.', color = 'red')




































 







[9]












lc.normalize().plot()
normalized_lc = lc.normalize()

































 






Binning Data

Simplify the data

Take multiple data points -> take them to one data point

Get rid of noise

Bin width is set (now) at 15 minutes, takes the average of all data points in each bin [width]

The larger the bin size, the less shape is preserved, and the less data is available to be manipulated


[10]














bin_time = 15/24/60 # LK uses day units, this is 15 minutes over 24 hours
#lc_collection_binned = lc_collection.bin(bin_time)
#lc_collection_binned.plot()

































 






Plotting region

Allows us to plot multiple data sets on one figure/graph


[11]















fig, ax = plt.subplots(figsize = (10, 5))
lc_collection.plot(ax = ax, linewidth = 0, marker = 'o', color = 'gold', markerSize = 1)
lc_collection_binned.plot(ax = ax, linewidth = 0, marker = 'o', color = 'black', markerSize = 1)






























AttributeError: 'Line2D' object has no property 'markerSize'
Show error details 
Search on Stack Overflow



 






Phase Folding

Fold different periods of separation of transit events onto each other 

`t0` -> first transit point

if (period == wrong) { lightDips != lineUp } /#/ Basic (if `t0` is wrong)





















!pip install lightkurve

import matplotlib.pyplot as plt
import lightkurve as lk
TIC = 'TIC 55525572'






























Execution has been cancelled


This output has been hidden. Show it.





















available_data_select = lk.search_lightcurve(TIC, author = 'SPOC')[0:9] # Query data, select data from mutliple sectors (e.g. all availablke first year TESS data)
lc_collection = available_data_select.download_all().stitch()






























Execution has been cancelled




 




















fig, ax = plt.subplots(figsize = (8,4))
lc_collection.plot(ax = ax, linewidth = 0, marker = 'o', color = 'purple', markersize = 1, alpha = 0.7)






























Execution has been cancelled




 










































period = 83.8979
t0 = 2125.847

lc_phased = lc_collection.fold(period = period, epoch_time = t0)
lc_phased.plot(linewidth = 0, color = 'gold', marker = '.', markersize = 1, alpha = 0.7)

lc_phased_binned = lc_phased.bin(15/24/60)
fig, ax = plt.subplots(figsize = (8,5)) # defines a plotting region to plot multiple data sets
lc_phased.plot(ax = ax, marker = '.', linewidth = 0, color = 'blue', alpha = 0.4, markersize = 3, label = 'unbinned')
lc_phased_binned.plot(ax = ax, marker = 'o', linewidth = 0, color = 'purple', alpha = 0.8, markersize = 6, label = 'binned')

plt.xlim(-2, 2) # upper and lower limit for x-axis, zooming in from -40, 40 wide to -2, 2 wide
plt.ylim(0.996, 1.004) 






























Execution has been cancelled




 






Determining planet stats

Use the fraction of light from the star being blocked out (in the dip) and compare that with the size of the parent star (make sure to account for things like ASB stars, but assume most to all stars observed will be main sequence).


Area of light -> area of planet (sphere -> disk): 

Area of star:


transitDepth=(rPlanetrStar)2transitDepth = ({rPlanet \over rStar})^2
transitDepth=(rStar


rPlanet
​)2

rPlanet=transitDepth×rStarrPlanet = \sqrt{transitDepth} \times rStar
rPlanet=transitDepth


​×rStar






































import matplotlib.pyplot as plt
import lightkurve as lk

plt.axhline(0.9988) # look at graph
plt.xlim(-2, 2) # upper and lower limit for x-axis, zooming in from -40, 40 wide to -2, 2 wide
plt.ylim(0.996, 1.004)

# https://deepnote.com/workspace/star-sailors-49d2efda-376f-4329-9618-7f871ba16007/project/Anomaly-Interaction-ab6b31e5-13c3-4949-af38-1197d00bd4d1
# https://deepnote.com/workspace/star-sailors-49d2efda-376f-4329-9618-7f871ba16007/project/lightkurvehandler-dca7e16c-429d-42f1-904d-43898efb2321
# https://deepnote.com/workspace/star-sailors-49d2efda-376f-4329-9618-7f871ba16007/project/Star-Sailors-Light-Curve-Plot-b4c251b4-c11a-481e-8206-c29934eb75da






























Execution has been cancelled




 


























!pip install astropy
from astropy import units as u
import numpy as np
import matplotlib.pyplot as plt
import lightkurve as lk




































Execution has been cancelled




 






















transit_depth = 1 - 0.9988
R_star = 2.04354 * u.Rsun # exofop.ipac.caltech.edu/tess/target.php?id=TIC * radius of [our] sun
r_pl_solar_radius = np.sqrt(transit_depth) * R_star






























Execution has been cancelled




 


















r_pl_solar_radius






























Execution has been cancelled




 






Some simple ideas/notes very briefly:

Move the plot into an interactive format (like a Zooniverse frontend fork) and look at where users "click" on "curves", save that to a DB

Connect DeepNote to Umbrel? -> https://b4c251b4-c11a-481e-8206-c29934eb75da.deepnoteproject.com

Spectroscopy -> https://tess.mit.edu/followup/



[Demo] Postgres
Saved to variablevisits 
 

































select
    visits.user_id,
    visits.visited_at,
    users.signed_up_at,
    users_ab.variant
from
    visits
    left join users on visits.user_id = users.user_id
    inner join users_ab on visits.user_id = users_ab.user_id




































Execution has been cancelled




 






Lightweight Curve Manipulation



















!pip install lightkurve

import matplotlib.pyplot as plt
import lightkurve as lk






























Execution has been cancelled




 






















TIC = 'TIC 284475976'
sector_data = lk.search_lightcurve(TIC, author = 'SPOC', sector = 23)
sector_data






























Execution has been cancelled




 




















lc = sector_data.download()
lc.plot






























Execution has been cancelled




 


















lc.plot(linewidth = 0, marker = '.', color = 'lightcyan', alpha = 0.8)






























Execution has been cancelled




 






Conversion Rate




We need to check whether one variant resulted in a higher conversion rate than another. Let's start with a time series.

























visits['registered'] = visits.signed_up_at.notna()
visits = visits.sort_values('visited_at').drop_duplicates('user_id',keep='first').copy()

conversion = visits.copy()
conversion['week'] = conversion.visited_at.dt.tz_localize(None).dt.to_period('W').dt.to_timestamp()
conversion = conversion.groupby(['variant','week']).registered.value_counts(normalize=True,dropna=False).reset_index(name='conversion')
conversion = conversion.loc[conversion.registered == True]




































Execution has been cancelled




 







Visualize data from
conversion








Refresh

Data

Format

Display




Type

Line chart





X Axis




TIME




week







Y Axis




#




conversion







Swap axes

Presentation
How to group?
Color




#




variant










Size
Select column











Hide sidebar

Layers


Line








Here is the overall conversion rate for each variant:



This code has been hidden. Show it.



Execution has been cancelled




 






Significance Test

It looks like Variant B has a higher conversion rate than Variant A. We need to make sure these results are significant.



This code has been hidden. Show it.



Execution has been cancelled




 






Retention Rate

We also want to know whether one variant of the website results in higher retention than another variant. For this, let's look at the weekly sessions of our users.



[Demo] Postgres
Saved to variablesessions_weekly 
 

































select
    sessions.user_id,
    date_trunc('week',users.signed_up_at) as signed_up_at_week,
    floor(extract('day' from session_started_at - signed_up_at)/7) as week, -- The number of weeks that passed since the user signed up
    users_ab.variant
from
    sessions
    left join users on sessions.user_id = users.user_id
    inner join users_ab on sessions.user_id = users_ab.user_id






























Execution has been cancelled


This output has been hidden. Show it.









This code has been hidden. Show it.



Execution has been cancelled




 







Visualize data from
retention








Refresh

Data

Format

Display




Type

Line chart





X Axis




#




Week



Y Axis




#




Retention




Swap axes

Presentation
How to group?
Color




#




variant










Size
Select column











Hide sidebar

Layers


Line








Here is the Week-4 Retention for each variant.













retention.loc[retention.Week == 4][['variant','Retention']].reset_index(drop=True)






























Execution has been cancelled




 






Significance Test

Once again, check that the difference in retention is significance.



This code has been hidden. Show it.



Execution has been cancelled




 








<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b4c251b4-c11a-481e-8206-c29934eb75da' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>